In [1]:
from collections import defaultdict, namedtuple
import itertools

# Data structures
Item = namedtuple('Item', ['name', 'price', 'slot_size'])
Transaction = namedtuple('Transaction', ['items'])
IndexEntry = namedtuple('IndexEntry', ['itemset', 'support', 'price', 'net_revenue_per_slot'])

class STUIndex:
    def __init__(self, max_level, lambda_val):
        self.max_level = max_level
        self.lambda_val = lambda_val
        self.levels = [[] for _ in range(max_level + 1)]

def calculate_metrics(itemset, transactions, items):
    support = sum(1 for t in transactions if set(itemset).issubset(set(t.items)))
    price = sum(items[item].price for item in itemset)
    slot_size = sum(items[item].slot_size for item in itemset)
    net_revenue = support * price
    net_revenue_per_slot = net_revenue / slot_size if slot_size > 0 else 0
    return support, price, net_revenue, net_revenue_per_slot

def build_stu_index(transactions, items, max_level, lambda_val, revenue_threshold):
    stu_index = STUIndex(max_level, lambda_val)
    
    # Level 1
    level_1_items = [(item,) for item in items]
    level_1_metrics = [calculate_metrics(itemset, transactions, items) for itemset in level_1_items]
    level_1_entries = [
        IndexEntry(itemset, support, price, net_revenue_per_slot)
        for itemset, (support, price, _, net_revenue_per_slot) in zip(level_1_items, level_1_metrics)
        if net_revenue_per_slot >= revenue_threshold
    ]
    stu_index.levels[1] = sorted(level_1_entries, key=lambda x: x.net_revenue_per_slot, reverse=True)[:lambda_val]
    
    # Higher levels
    for level in range(2, max_level + 1):
        candidate_itemsets = set()
        for lower_level_itemset in stu_index.levels[level - 1]:
            for item in items:
                new_itemset = tuple(sorted(set(lower_level_itemset.itemset) | {item}))
                if len(new_itemset) == level:
                    candidate_itemsets.add(new_itemset)
        
        level_metrics = [calculate_metrics(itemset, transactions, items) for itemset in candidate_itemsets]
        level_entries = [
            IndexEntry(itemset, support, price, net_revenue_per_slot)
            for itemset, (support, price, _, net_revenue_per_slot) in zip(candidate_itemsets, level_metrics)
            if net_revenue_per_slot >= revenue_threshold
        ]
        stu_index.levels[level] = sorted(level_entries, key=lambda x: x.net_revenue_per_slot, reverse=True)[:lambda_val]
    
    return stu_index

def tipds_placement(stu_index, num_premium_slots):
    placement = []
    remaining_slots = num_premium_slots
    level = 1
    top_k = 1

    while remaining_slots > 0:
        if level > stu_index.max_level:
            level = 1
            top_k += 1

        if top_k > len(stu_index.levels[level]):
            level += 1
            continue

        itemset = stu_index.levels[level][top_k - 1].itemset
        itemset_size = sum(items[item].slot_size for item in itemset)

        if itemset_size <= remaining_slots:
            placement.append(itemset)
            remaining_slots -= itemset_size

        level += 1

    return placement

# Example usage
items = {
    'A': Item('A', 7, 3),
    'B': Item('B', 2, 2),
    'C': Item('C', 6, 1),
    'D': Item('D', 1, 2),
    'E': Item('E', 3, 4),
    'F': Item('F', 1, 3),
    'G': Item('G', 5, 2),
    'H': Item('H', 4, 5),
    'I': Item('I', 3, 2)
}

transactions = [
    Transaction(['A', 'D']),
    Transaction(['B', 'C', 'I', 'F']),
    Transaction(['A', 'C', 'G']),
    Transaction(['A', 'B', 'C', 'G', 'H']),
    Transaction(['A', 'C', 'G', 'I'])
]

max_level = 4
lambda_val = 5
revenue_threshold = 6.54
num_premium_slots = 20

stu_index = build_stu_index(transactions, items, max_level, lambda_val, revenue_threshold)
placement = tipds_placement(stu_index, num_premium_slots)

print("STU Index:")
for level, entries in enumerate(stu_index.levels):
    if entries:
        print(f"Level {level}:")
        for entry in entries:
            print(f"  {entry}")

print("\nTIPDS Placement:")
print(placement)

KeyboardInterrupt: 

In [9]:
import pandas as pd
from collections import defaultdict

class STUIndex:
    def __init__(self, max_levels, lambda_value):
        self.max_levels = max_levels
        self.lambda_value = lambda_value
        self.index = defaultdict(list)

    def build_index(self, items, transactions):
        # Step 1: Build level 1 of the index
        self._build_level_1(items, transactions)

        # Step 2: Build higher levels of the index
        for level in range(2, self.max_levels + 1):
            self._build_higher_level(level, items)

    def _build_level_1(self, items, transactions):
        item_stats = defaultdict(lambda: {'frequency': 0, 'revenue': 0})

        # Calculate frequency and revenue for each item
        for transaction in transactions:
            for item in transaction:
                item_stats[item]['frequency'] += 1
                item_stats[item]['revenue'] += items[item]['price']

        # Calculate net revenue per slot and filter top-λ items
        level_1_items = []
        for item, stats in item_stats.items():
            net_revenue_per_slot = stats['revenue'] / items[item]['slot_size']
            level_1_items.append((item, stats['frequency'], stats['revenue'], net_revenue_per_slot))

        # Sort by net revenue per slot and keep top-λ items
        level_1_items.sort(key=lambda x: x[3], reverse=True)
        self.index[1] = level_1_items[:self.lambda_value]

    def _build_higher_level(self, level, items):
        candidate_itemsets = self._generate_candidate_itemsets(level, items)
        
        # Calculate metrics for new itemsets
        itemset_stats = []
        for itemset in candidate_itemsets:
            frequency = self._calculate_frequency(itemset)
            revenue = sum(items[item]['price'] for item in itemset)
            slot_size = sum(items[item]['slot_size'] for item in itemset)
            net_revenue_per_slot = revenue / slot_size
            itemset_stats.append((itemset, frequency, revenue, net_revenue_per_slot))

        # Sort by net revenue per slot and keep top-λ itemsets
        itemset_stats.sort(key=lambda x: x[3], reverse=True)
        self.index[level] = itemset_stats[:self.lambda_value]

    def _generate_candidate_itemsets(self, level, items):
        if level == 2:
            return self._generate_2_itemsets(items)
        else:
            return self._generate_k_itemsets(level, items)

    def _generate_2_itemsets(self, items):
        level_1_items = [item[0] for item in self.index[1]]
        return [(i, j) for i in level_1_items for j in level_1_items if i < j]

    def _generate_k_itemsets(self, k, items):
        prev_level_itemsets = [itemset[0] for itemset in self.index[k-1]]
        level_1_items = [item[0] for item in self.index[1]]
        
        candidate_itemsets = []
        for itemset in prev_level_itemsets:
            for item in level_1_items:
                if item > itemset[-1]:
                    new_itemset = itemset + (item,)
                    if all(tuple(sorted(new_itemset[:i] + new_itemset[i+1:])) in prev_level_itemsets 
                           for i in range(len(new_itemset))):
                        candidate_itemsets.append(new_itemset)
        
        return candidate_itemsets

    def _calculate_frequency(self, itemset):
        # This is a placeholder. In a real implementation, you would calculate
        # the actual frequency of the itemset in the transaction database.
        return 1

    def get_top_itemsets(self, slot_size):
        return self.index.get(slot_size, [])

# Example usage
items = {
    'A': {'price': 10, 'slot_size': 1},
    'B': {'price': 15, 'slot_size': 2},
    'C': {'price': 20, 'slot_size': 1},
    'D': {'price': 25, 'slot_size': 3},
}

transactions = [
    ['A', 'B', 'C'],
    ['B', 'C', 'D'],
    ['A', 'C', 'D'],
    ['A', 'B', 'D'],
]

stu_index = STUIndex(max_levels=4, lambda_value=5)
stu_index.build_index(items, transactions)

# Retrieve top itemsets for slot size 2
# top_itemsets = stu_index.get_top_itemsets(2)
print("Top itemsets for slot size 1:", stu_index.get_top_itemsets(1))
print("Top itemsets for slot size 2:", stu_index.get_top_itemsets(2))
print("Top itemsets for slot size 3:", stu_index.get_top_itemsets(3))
# print("Top itemsets for slot size 4:", stu_index.get_top_itemsets(4))


Top itemsets for slot size 1: [('C', 3, 60, 60.0), ('A', 3, 30, 30.0), ('D', 3, 75, 25.0), ('B', 3, 45, 22.5)]
Top itemsets for slot size 2: [(('A', 'C'), 1, 30, 15.0), (('B', 'C'), 1, 35, 11.666666666666666), (('C', 'D'), 1, 45, 11.25), (('A', 'D'), 1, 35, 8.75), (('A', 'B'), 1, 25, 8.333333333333334)]
Top itemsets for slot size 3: [(('A', 'B', 'C'), 1, 45, 11.25), (('A', 'C', 'D'), 1, 55, 11.0)]
